In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import Ridge, LassoCV
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from scipy.stats import spearmanr, pearsonr

# === Toy predictors ===
Nsamps = 500

# Initialize predictors
dff = np.zeros(Nsamps)
dff[0] = 1

cues = np.zeros(Nsamps)
cues[0] = 1

GO = np.zeros(Nsamps)
GO[0] = 1

NOGO = np.zeros(Nsamps)
NOGO[0] = 1

rew = np.zeros(Nsamps)
rew[0] = 1

norew = np.zeros(Nsamps)
norew[0] = 1

licks = np.zeros(Nsamps)
licks[0] = 1

fluo = np.zeros(Nsamps)

# Simulate data reading
# Replace these assignments with real data loading in practice
dff = dff[10:]  # simulate dff = dff_database[11,:]
cues = cues
GO = GO
NOGO = NOGO
rew = rew
norew = norew
licks = licks

# Create lagged predictors
def create_lagged_matrix(signal, pre=10, post=2):
    lags = []
    for lag in range(post, 0, -1):
        shifted = np.roll(signal, -lag)
        shifted[-lag:] = 0
        lags.append(shifted)
    lags.append(signal)
    for lag in range(1, pre+1):
        shifted = np.roll(signal, lag)
        shifted[:lag] = 0
        lags.append(shifted)
    return np.column_stack(lags)

cue_lags = create_lagged_matrix(cues, pre=10, post=2)
go_lags = create_lagged_matrix(GO, pre=10, post=2)
nogo_lags = create_lagged_matrix(NOGO, pre=10, post=2)
rew_lags = create_lagged_matrix(rew, pre=10, post=3)
norew_lags = create_lagged_matrix(norew, pre=10, post=3)
licks_lags = create_lagged_matrix(licks, pre=10, post=4)

# Trial-based covariates
trial_length = 41
trial_num = np.ceil(np.arange(1, Nsamps + 1) / trial_length)
time_in_trial = np.arange(1, Nsamps + 1) - trial_length * (trial_num - 1)

# Design matrix
X = np.column_stack([cue_lags, go_lags, nogo_lags,
                     rew_lags, norew_lags, licks_lags,
                     fluo, trial_num, time_in_trial])

# === Ridge Regression ===
alphas = np.linspace(0.001, 0.1, 100)
ridge_model = Ridge(alpha=1.0)
ridge_model.fit(X, dff)
b_ridge = ridge_model.coef_

# === Lasso Regression with Cross-Validation ===
lasso_model = LassoCV(cv=10, alphas=None, max_iter=10000)
lasso_model.fit(X, dff)
b_lasso = lasso_model.coef_
intercept = lasso_model.intercept_
opt_alpha = lasso_model.alpha_

# Predicted dF/F
dff_hat = X @ b_lasso + intercept

# Correlation analysis
rho_s, pval_s = spearmanr(dff, dff_hat)
rho_p, pval_p = pearsonr(dff, dff_hat)

# Print coefficients grouped by predictor type
cue_weights = np.sum(np.abs(b_lasso[0:13]))
go_weights = np.sum(np.abs(b_lasso[13:26]))
nogo_weights = np.sum(np.abs(b_lasso[26:39]))
rew_weights = np.sum(np.abs(b_lasso[39:53]))
norew_weights = np.sum(np.abs(b_lasso[53:67]))
licks_weights = np.sum(np.abs(b_lasso[67:82]))

coeff_weights = np.array([cue_weights, go_weights, nogo_weights,
                          rew_weights, norew_weights, licks_weights])

coeff_weights /= np.max(coeff_weights)

print(f"Cues: {coeff_weights[0]:.5f}, GO: {coeff_weights[1]:.5f}, NOGO: {coeff_weights[2]:.5f}, "
      f"Reward: {coeff_weights[3]:.5f}, No Reward: {coeff_weights[4]:.5f}, Licks: {coeff_weights[5]:.5f}")

# Plot predicted vs actual
plt.figure()
plt.scatter(dff, dff_hat, alpha=0.5)
plt.plot([dff.min(), dff.max()], [dff.min(), dff.max()], 'r--')
plt.xlabel('Actual dF/F')
plt.ylabel('Predicted dF/F')
plt.title('Lasso: Predicted vs Actual')
plt.grid(True)
plt.show()

# Print correlation results
print(f"Spearman correlation: rho = {rho_s:.3f}, p = {pval_s:.3g}")
print(f"Pearson correlation: rho = {rho_p:.3f}, p = {pval_p:.3g}")
